Importación de librerias

In [1]:
import urllib.request, json
import pandas as pd
import requests as re

Función para realizar el primer scrape de la API para transformar el json en un DataFrame y por úlitmo, exportarlo en un .csv. 
La función requiere de tres variables:
* item = string con el término de busqueda que queremos realizar. ex: "smartphone", "Apple", "Chromecast".
* category = string con el category id que querramos buscar. ex: "MLA352001" para la categoría "Media  Streaming"
* csvname = el nombre que le queramos dar al archivo *.csv, solo el nombre no hay que agregar la extensión de archivo.

In [2]:
def scrape_to_df(item, category, csvname):
    with urllib.request.urlopen("https://api.mercadolibre.com/sites/MLA/search?category={0}&q={1}#json".format(category, item)) as url:
        data = json.loads(url.read().decode())
        df = pd.json_normalize(data["results"])
        df = pd.DataFrame(df)
        df.to_csv("{0}.csv".format(csvname))
        return df

Realizamos el primer scrape con los DataFrames de los items a analizar. Al llamar la función con una variable, la misma se convierte en un DataFrame.

In [3]:
chromecast_df = scrape_to_df("chromecast", "MLA352001", "chromecast")
xiaomi_df = scrape_to_df("xiaomi", "MLA352001", "xiaomi_tv")
appletv_df = scrape_to_df("apple&tv", "MLA352001", "appletv")
chromecast_df.head(1)

,id,site_id,title,price,sale_price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,...,seller_address.zip_code,seller_address.country.id,seller_address.country.name,seller_address.state.id,seller_address.state.name,seller_address.city.id,seller_address.city.name,seller_address.latitude,seller_address.longitude,differential_pricing.id
0,MLA870831824,MLA,Google Chromecast Full Hd Carbón,5699.05,None,ARS,302,20054,buy_it_now,gold_special,...,,AR,Argentina,AR-C,Capital Federal,QVItQ1BhcnF1ZSBDaGFz,Parque Chas,,,NaN


Segunda función para realizar la consulta de los items conseguidos en los DataFrames. La segunda función tiene dos variables:
* ids = se llama a la columna "id" con df["id"], esto permite iterar en un for loop a cada valor item_id.
* df_out = nombre del DataFrame donde almacenamos los datos. Primero hay que crear el Dataframe vacio con df = pd.DataFrame()

In [4]:
def scrape_items (ids, df_out):
    for i in ids:
        iter_get = re.get("https://api.mercadolibre.com/items/{0}".format(i))
        response = iter_get.json() 
        df = pd.json_normalize(response)
        df_out = df_out.append(df)
    return df_out

Para utilizar la función correctamente debemos seguir los siguientes pasos:
1. crear el primer DataFrame vacio
2. llamar a la funcion

In [5]:
#Chromecast#
chrome_items = pd.DataFrame()
chrome_items = scrape_items(chromecast_df["id"], chrome_items)
#Xiami#
xiaomi_items = pd.DataFrame()
xiaomi_items = scrape_items(xiaomi_df["id"], xiaomi_items)
#Apple TV#
appletv_items = pd.DataFrame()
appletv_items = scrape_items(appletv_df["id"], appletv_items)

Por último, exportamos los DataFrames en formato csv.

In [6]:
chrome_items.to_csv("chromecast_items.csv")
xiaomi_items.to_csv("xiaomi_items.csv")
appletv_items.to_csv("appletv_items.csv")